<a href="https://colab.research.google.com/github/YBen1/PandaPower_Basics/blob/main/Lesson8_simple_state_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lesson 8


In [1]:
!pip install pandapower

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pytest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandapower.test
import pandapower as pp
import pandapower.networks as nw
import pandapower.topology as top
import pandapower.plotting as plot
import matplotlib.pyplot as plt
import pandapower.converter
import pandapower.estimation
import networkx as nx

In [4]:
net = pp.create_empty_network()


In [5]:
# create buses
b1 = pp.create_bus(net, vn_kv=1., name='bus1', index =1)
b2 = pp.create_bus(net, vn_kv=1., name='bus2', index =2)
b3 = pp.create_bus(net, vn_kv=1., name='bus3', index =3)

In [6]:
pp.create_ext_grid(net, b1)

0

In [7]:
# create 110 kV lines
l1 = pp.create_line_from_parameters(net,1,2,1, r_ohm_per_km=.01, x_ohm_per_km=.03, c_nf_per_km= 0., max_i_ka =1)
l2 =  pp.create_line_from_parameters(net,1,3,1, r_ohm_per_km=.02, x_ohm_per_km=.05, c_nf_per_km= 0., max_i_ka =1)
l3 = pp.create_line_from_parameters(net,2,3,1, r_ohm_per_km=.03, x_ohm_per_km=.08, c_nf_per_km= 0., max_i_ka =1)

In [8]:
# bus voltages
pp.create_measurement(net,"v", "bus", 1.006, 0.004, b1)
pp.create_measurement(net,"v", "bus", .968, 0.004, b2)

1

In [10]:
#bus p,q
pp.create_measurement(net, "p", "bus", 0.501, 0.01, element=b2)         # P at bus 2
pp.create_measurement(net, "q", "bus", 0.286, 0.01, element=b2)         # Q at bus 2
net.measurement

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,1,1.006,0.004,None
1,None,v,bus,2,0.968,0.004,None
2,None,p,bus,2,0.501,0.010,None
3,None,q,bus,2,0.286,0.010,None


In [11]:
pp.create_measurement(net, "p", "line", 0.888, 0.008, element=l1, side=b1)  # Pline (bus 1 -> bus 2) at bus 1
pp.create_measurement(net, "p", "line", 1.173, 0.008, element=l2, side=b1)  # Pline (bus 1 -> bus 3) at bus 1
pp.create_measurement(net, "q", "line", 0.568, 0.008, element=l1, side=b1)  # Qline (bus 1 -> bus 2) at bus 1
pp.create_measurement(net, "q", "line", 0.663, 0.008, element=l2, side=b1)  # Qline (bus 1 -> bus 3) at bus 1
net.measurement


,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,1,1.006,0.004,None
1,None,v,bus,2,0.968,0.004,None
2,None,p,bus,2,0.501,0.010,None
3,None,q,bus,2,0.286,0.010,None
4,None,p,line,0,0.888,0.008,1
5,None,p,line,1,1.173,0.008,1
6,None,q,line,0,0.568,0.008,1
7,None,q,line,1,0.663,0.008,1


In [12]:
pp.create_measurement(net, "p", "line", 1.0, 0.008, element=l1, side=b3)  # Pline (bus 1 -> bus 2) at bus 3
net.measurement

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,1,1.006,0.004,None
1,None,v,bus,2,0.968,0.004,None
2,None,p,bus,2,0.501,0.010,None
3,None,q,bus,2,0.286,0.010,None
4,None,p,line,0,0.888,0.008,1
5,None,p,line,1,1.173,0.008,1
6,None,q,line,0,0.568,0.008,1
7,None,q,line,1,0.663,0.008,1
8,None,p,line,0,1.000,0.008,3


In [13]:
from pandapower.estimation import remove_bad_data
import numpy as np

success_rn_max = remove_bad_data(net, init='flat', rn_max_threshold=3.0)
print(success_rn_max)



True


In [14]:
net.res_bus_est.vm_pu

1    0.999629
2    0.974156
3    0.943890
Name: vm_pu, dtype: float64

In [15]:
net.res_bus_est.va_degree

1    0.000000
2   -1.247547
3   -2.745717
Name: va_degree, dtype: float64

In [16]:
net.res_bus_est.p_mw

1   -2.064016
2    0.495975
3    1.514221
Name: p_mw, dtype: float64

In [17]:
net.res_bus_est.q_mvar

1   -1.226440
2    0.297750
3    0.787529
Name: q_mvar, dtype: float64

In [18]:
net.res_line_est.p_from_mw

0    0.892992
1    1.171024
2    0.385912
Name: p_from_mw, dtype: float64

In [19]:
net.res_line_est.q_from_mvar

0    0.558822
1    0.667619
2    0.227756
Name: q_from_mvar, dtype: float64